# **Metabolism Analysis of T. muris and C. elegans**

---

#### **Students Names:** Abril Iglesias Jiménez, Patricia Sánchez Mengual, Carla Licer Otero, Laia Montenegro Domenech, Esther Batalla Royo and Andrea Lauro
#### **Date:** [Fecha de entrega]
#### **Course:**  Computational Biology and Biomedical Data Analysis - Universitat Rovira i Virgili

---
## **1. Introduction**

Soil-transmitted helminth infections cause health problems such as diarrhea, abdominal pain, malnutrition, and growth retardation. Human trichuriasis, caused by *Trichuris trichiura*, is one of the most common infections.

Due to the difficulty of studying this species directly in the laboratory, *Trichuris muris*, a mouse parasite with very similar biological and pathological characteristics, is used as an experimental model.

The main objective of this project is to analyze the metabolism of *T. muris*, compare it with *Caenorhabditis elegans*, and identify essential genes that could serve as potential therapeutic targets for the treatment of whipworm infections.

## **2. Genome-Scale Metabolic Models Acquisition**

In this section, we will obtain and load the genome-scale metabolic models (GEMs) of *Trichuris muris* and *Caenorhabditis elegans* from public GitHub repositories.  


## Import libraries

In [23]:
import cobra
import gzip
from cobra.io import load_model
from Bio import SeqIO, Align, pairwise2
import pandas as pd
import time

## Set file paths

In [24]:
tmuris_xml = '/opt/notebooks/final_project/Genomicscale/Tmuris.xml'
celegans_xml = '/opt/notebooks/final_project/Genomicscale/Celegans.xml'
tmuris_protein_fa = '/opt/notebooks/final_project/AAsequence/Tmuris.protein.fa.gz'
celegans_protein_fa = '/opt/notebooks/final_project/AAsequence/Celegans.protein.fa.gz'

In [25]:
# Load the genome-scale metabolic model of T. muris
T_muris_GS = cobra.io.read_sbml_model(tmuris_xml)

# Load the genome-scale metabolic model of C. elegans
C_elegans_GS = cobra.io.read_sbml_model(celegans_xml)

display(T_muris_GS)
display(C_elegans_GS)

https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Name,iTMU798
Memory address,77bc59f01af0
Number of metabolites,930
Number of reactions,1219
Number of genes,800
Number of groups,0
Objective expression,1.0*WBPTMR0091 - 1.0*WBPTMR0091_reverse_cd110
Compartments,"Mitochondria, Cytosol, Extracellular Space"


Name,WormGEM
Memory address,77bc59f031a0
Number of metabolites,8175
Number of reactions,11936
Number of genes,1604
Number of groups,153
Objective expression,1.0*MAR00021 - 1.0*MAR00021_reverse_97974
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


The genome-scale model **iTMU798** of *T. muris* contains **930 metabolites**, **1219 reactions**, and **800 genes**, which aligns well with the original publication. The model includes three compartments (cytosol, mitochondria, and extracellular space). 

The genome-scale model **WormGEM** of *C. elegans* includes **8175 metabolites**, **11,936 reactions**, and **1604 genes**. It defines **153 metabolic groups**. The model icludes multiple cellular compartments, such as cytosol, nucleus, mitochondria, lysosome, endoplasmic reticulum, and others.

## **3. Protein Sequences Acquisition**

In this section, we obtain and load the amino acid sequences for all proteins from *Trichuris muris* and *Caenorhabditis elegans*.  
The sequences are retrieved from WormBase Parasite in FASTA format (.fa.gz files).


In [26]:
# Function to load protein sequences from a compressed .fa.gz file
def load_protein_sequences(file_path):
    with gzip.open(file_path, "rt") as handle:  # Open the .gz file in text mode
        records = list(SeqIO.parse(handle, "fasta"))  # Parse sequences in FASTA format
    return records  # Return the list of sequence records

# Load protein sequences for T. muris
t_muris_proteins = load_protein_sequences(tmuris_protein_fa)

# Load protein sequences for C. elegans
c_elegans_proteins = load_protein_sequences(celegans_protein_fa)

# Print the number of protein sequences loaded
print("T. muris proteins loaded", len(t_muris_proteins), " sequences.")
print("C. elegans proteins loaded:", len(c_elegans_proteins), "sequences.")
print("\n")

# Display information about the first T. muris protein
print("First T. muris protein:")
print("ID:", t_muris_proteins[0].id)  # Show protein ID
print("Description:", t_muris_proteins[0].description)  # Show description if available
print("Sequence:", t_muris_proteins[0].seq)  # Show the amino acid sequence

print("\n-------------------------------------------------------------------------------\n")

# Display information about the first C. elegans protein
print("First C. elegans protein:")
print("ID:", c_elegans_proteins[0].id)  
print("Description:", c_elegans_proteins[0].description)  
print("Sequence:", c_elegans_proteins[0].seq) 


T. muris proteins loaded 14995  sequences.
C. elegans proteins loaded: 28577 sequences.


First T. muris protein:
ID: TMUE_0000000001
Description: TMUE_0000000001 wormpep=TMP03343 gene=WBGene00295949 status=Predicted
Sequence: MVFPSFLLSSKTQRGIQKASKWWDESTKLAEAIVLGALGPSPATCNRCKELKIWWALFQPALPVAPSQ

-------------------------------------------------------------------------------

First C. elegans protein:
ID: 2L52.1a
Description: 2L52.1a wormpep=CE32090 gene=WBGene00007063 status=Confirmed uniprot=A4F336 insdc=CCD61130.1 product="C2H2-type domain-containing protein"
Sequence: MSMVRNVSNQSEKLEILSCKWVGCLKSTEVFKTVEKLLDHVTADHIPEVIVNDDGSEEVVCQWDCCEMGASRGNLQKKKEWMENHFKTRHVRKAKIFKCLIEDCPVVKSSSQEIETHLRISHPINPKKERLKEFKSSTDHIEPTQANRVWTIVNGEVQWKTPPRVKKKTVIYYDDGPRYVFPTGCARCNYDSDESELESDEFWSATEMSDNEEVYVNFRGMNCISTGKSASMVPSKRRNWPKRVKKRLSTQRNNQKTIRPPELNKNNIEIKDMNSNNLEERNREECIQPVSVEKNILHFEKFKSNQICIVRENNKFREGTRRRRKNSGESEDLKIHENFTEKRRPIRSCKQNISFYEMDGDIEEFEVFFDTPTKSKKVLLDIYSAKKMPKIEVEDSLVNKFHSKRPSRACRVLGSMEEVPFD

From these results, we can confirm that both proteomes were correctly loaded and are consistent with the expected data from WormBase Parasite. *T. muris* includes 14995 protein sequences, and *C. elegans* includes 28577. 


## **4. Identification of Orthologous Enzymes via Pairwise Sequence Alignment**

In this section, we focus on identifying orthologous enzymes between *Trichuris muris* and *Caenorhabditis elegans*.  We begin by extracting the protein-coding genes involved in enzymatic reactions from the *T. muris* metabolic model.

We then perform pairwise sequence alignment between each *T. muris* enzymatic protein and all proteins in the *C. elegans* to identify the best-matching orthologs based on sequence similarity.  
This analysis allows us to examine whether the metabolic function of each enzyme is conserved in *C. elegans*, or if there are significant differences that could indicate functional divergence or specialization.



In [27]:
#SEARCHING ID OF THE ENZYMATIC GENES

# Initialize an empty list to store enzymatic gene IDs
enzymatic_genes_tmuris =[]
    
# Loop through each gene object in the T. muris model, .genes gets the enzymatic proteins
for gene in T_muris_GS.genes:
    enzymatic_genes_tmuris.append(gene.id) #adds to a list the id of each gene
    
# Print results
print("Number of enzymatic genes in T. muris model: ", len(enzymatic_genes_tmuris))
print("Example gene IDs: ", enzymatic_genes_tmuris[:5])

# Parámetros de alignment
match_score = 1.0
mismatch_score = -1.0
gap_open = -1.5
gap_extend = -0.5



output_file = "orthologs_blockwise.csv"




Number of enzymatic genes in T. muris model:  800
Example gene IDs:  ['TMUE_3000011849', 'TMUE_0000000186', 'TMUE_3000012793', 'TMUE_3000011199', 'TMUE_3000013102']


In [28]:
# Crear diccionario de secuencias de T. muris por ID
tmuris_seq_dict = {record.id: record for record in t_muris_proteins}

# Filtrar solo genes enzimáticos que tengan secuencia disponible
enzymatic_genes_with_seq = []
for gene_id in enzymatic_genes_tmuris:
    if gene_id in tmuris_seq_dict:
        enzymatic_genes_with_seq.append(tmuris_seq_dict[gene_id])

print(f"Genes enzimáticos de T. muris con secuencia disponible: {len(enzymatic_genes_with_seq)} / {len(enzymatic_genes_tmuris)}")


Genes enzimáticos de T. muris con secuencia disponible: 798 / 800


In [30]:
# Configuración previa
c_elegans_gene_ids = {gene.id for gene in C_elegans_GS.genes}
c_elegans_model_map = {gene.id: [rxn.id for rxn in gene.reactions] for gene in C_elegans_GS.genes}
min_len_ratio = 0.7
max_len_ratio = 1.4

# Inicialización
orthologs_detailed = []
total_blocks = len(enzymatic_genes_with_seq)
start_total = time.time()

# Loop principal
for block_num, tm_protein in enumerate(enzymatic_genes_with_seq, start=1):
    start_block = time.time()

    len_tm = len(tm_protein.seq)
    for ce_protein in c_elegans_proteins:
        len_ce = len(ce_protein.seq)

        # Filtrado por longitud
        if len_ce / len_tm < min_len_ratio or len_ce / len_tm > max_len_ratio:
            continue

        # Alineamiento rápido
        score_result = pairwise2.align.globalms(tm_protein.seq, ce_protein.seq, match_score, mismatch_score, gap_open, gap_extend, score_only=True)
        identity = score_result / len_tm

        if identity > 0.7:
            ce_id = ce_protein.id
            ce_in_model = ce_id in c_elegans_model_map
            ce_reactions = ";".join(c_elegans_model_map.get(ce_id, []))

            orthologs_detailed.append({
                "ID_T_muris": tm_protein.id,
                "ID_C_elegans": ce_id,
                "Score": score_result,
                "%_identity": round(identity * 100, 2),
                "In_C_elegans_Model": ce_in_model,
                "C_elegans_Reactions": ce_reactions,
                "Length_T_muris": len_tm,
                "Length_C_elegans": len_ce
            })

    # Progreso por bloque
    block_time = time.time() - start_block
    elapsed = time.time() - start_total
    avg_block_time = elapsed / block_num
    remaining_time = (total_blocks - block_num) * avg_block_time

    print(f"[{block_num}/{total_blocks}] proteínas T. muris - "
          f"Tiempo para este bloque: {block_time:.2f}s - "
          f"Total: {elapsed/60:.2f} min - Est. restante: {remaining_time/60:.2f} min")


[1/798] proteínas T. muris - Tiempo para este bloque: 16.61s - Total: 0.28 min - Est. restante: 220.64 min
[2/798] proteínas T. muris - Tiempo para este bloque: 15.77s - Total: 0.54 min - Est. restante: 214.79 min
[3/798] proteínas T. muris - Tiempo para este bloque: 15.28s - Total: 0.79 min - Est. restante: 210.51 min
[4/798] proteínas T. muris - Tiempo para este bloque: 8.70s - Total: 0.94 min - Est. restante: 186.47 min
[5/798] proteínas T. muris - Tiempo para este bloque: 17.79s - Total: 1.24 min - Est. restante: 196.01 min


KeyboardInterrupt: 

In [ ]:
df_orthologs = pd.DataFrame(orthologs_detailed)
df_orthologs.to_csv("orthologs_with_functions.csv", index=False)
print("Archivo guardado: orthologs_with_functions.csv")